# Kafka Producer: dealer #2

In [ ]:
pip install confluent_kafka --quiet

In [ ]:
from confluent_kafka import Producer, KafkaError
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
import string

topic_vehicles = "DEALER2_VEHICLES"
topic_users = "DEALER2_USERS"

a = AdminClient({'bootstrap.servers': 'kafka:9092'})
fs = a.create_topics([NewTopic(topic_vehicles, num_partitions=1, replication_factor=1),
                      NewTopic(topic_users, num_partitions=1, replication_factor=1)])

for t, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Topic {} created".format(t))
    except Exception as e:
        print("Failed to create topic {}: {}".format(t, e))

Generate and publish a new topic message every 20 to 40 seconds.

In [ ]:
producerconf = {'bootstrap.servers': 'kafka:9092'}
prod = Producer(producerconf)

In [ ]:
from csv import DictReader,reader
from datetime import datetime
import time
import random
import json

users = {}
registered_users = []
with open('/home/jovyan/data_generator/users.csv', 'r') as user_file:
    csv_reader = reader(user_file)
    header = next(csv_reader)
    for row in csv_reader:
        users[row[0]]=row[1]
        

# Kafka topic writing
for vehicle in iter(DictReader(open('/home/jovyan/data_generator/vehicles.csv', 'r'))):
    
    if vehicle["userID"] not in registered_users:
        uid = vehicle["userID"]
        registered_users.append(uid)
        user = {"userID": uid, "name": users[uid]}
        print("USER: {}".format(user))
        prod.produce(topic=topic_users, value=json.dumps(user))
    
    vehicle["ts"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print("VEHICLE: {}".format(vehicle))
    prod.produce(topic=topic_vehicles, value=json.dumps(vehicle))
    
    # Random data generation sleeping interval
    time.sleep(random.randint(20,40))